<a href="https://colab.research.google.com/github/shakyaroopal/FakeNews-Detection/blob/main/fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re#regular expression searching
from nltk.corpus import stopwords#
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [ ]:
#loading the dataset to pandas data set
news_dataset= pd.read_csv('/content/train3.csv')

In [ ]:
news_dataset.shape

(20788, 5)

In [ ]:
#print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnâ€™t Even See Comeyâ€™s...,Darrell Lucus,House Dem Aide: We Didnâ€™t Even See Comeyâ€™s...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#counting the number of missiong values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1956
text       116
label        0
dtype: int64

In [ ]:
#replacing null values with empty string
newsdatset=news_dataset.fillna('')

In [ ]:
#merging the author name and news title
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didnâ€™t Even...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20783    Jerome Hudson Rapper T.I.: Trump a â€™Poster C...
20784    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20785    Michael J. de la Merced and Rachel Abrams Macy...
20786    Alex Ansary NATO, Russia To Hold Parallel Exer...
20787              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20788, dtype: object


In [ ]:
#separating the data & label
X=news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']

In [ ]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didnâ€™t Even See Comeyâ€™s...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20783  20795  Rapper T.I.: Trump a â€™Poster Child For White...   
20784  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20785  20797  Macyâ€™s Is Said to Receive Takeover Approach ...   
20786  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20787  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:
Stemming is the process of reducing a word to its root word.

In [ ]:
port_stem=PorterStemmer()

NameError: ignored

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]'," ",str(content))
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20783    jerom hudson rapper trump poster child white s...
20784    benjamin hoffman n f l playoff schedul matchup...
20785    michael j de la merc rachel abram maci said re...
20786    alex ansari nato russia hold parallel exerci b...
20787                            david swanson keep f aliv
Name: content, Length: 20788, dtype: object


In [ ]:
#separating the data and label
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
Y.shape

(20788,)

In [ ]:
#Converting textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [ ]:
print(X)

  (0, 14503)	0.285532950967956
  (0, 12451)	0.2563559322747842
  (0, 8228)	0.3596591621222568
  (0, 7969)	0.2934714209511952
  (0, 7115)	0.24644012238774501
  (0, 6484)	0.2188517277067959
  (0, 4584)	0.23132260474156088
  (0, 3512)	0.26890626323127514
  (0, 3328)	0.3596591621222568
  (0, 2733)	0.24749251570659517
  (0, 2294)	0.3730640070474714
  (0, 243)	0.27025697398904697
  (1, 15530)	0.3071999806468903
  (1, 6311)	0.19637557716254525
  (1, 5081)	0.7142307304127277
  (1, 3297)	0.2649121159919582
  (1, 2597)	0.19727980379135363
  (1, 2050)	0.36691190807859464
  (1, 1748)	0.15518185616127605
  (1, 1376)	0.29855612766095285
  (2, 14437)	0.4178883183679331
  (2, 8890)	0.4931065145066445
  (2, 5519)	0.35058423461959465
  (2, 4972)	0.38760384996385094
  (2, 2866)	0.4571259055069088
  :	:
  (20785, 12124)	0.2531149416481719
  (20785, 11406)	0.2748085968111548
  (20785, 11213)	0.24637548851410743
  (20785, 9520)	0.08014200033255728
  (20785, 8859)	0.1730966416537195
  (20785, 8797)	0.2910342

Splitting the dataset to training and test data

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

Training the Model:Logistic Regresson

In [ ]:
model=LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

### Evaluation
Accuracy score

In [ ]:
#accuracy on training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [ ]:
print(training_data_accuracy)

0.9881539386650632


In [ ]:
#accuracy on test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [ ]:
print(test_data_accuracy)

0.9877344877344877


Making a Predictive System

In [ ]:
X_new=X_test[14]
prediction=model.predict(X_new)
print(prediction)
if(prediction[0]==0):
  print('Real news')
else:
  print('Fake news')

[1]
Fake news


In [ ]:
print(Y_test[14])

1
